# Limpieza de Datos del Dataset IGAC

Este notebook procesa el dataset completo aplicando funciones de limpieza.

## Objetivos
1. Procesar el dataset completo en chunks
3. Validar y corregir fechas
4. Limpiar valores numéricos
5. Manejar valores nulos
6. Eliminar duplicados
7. Guardar datos limpios en formato Parquet

In [20]:
import sys
from pathlib import Path
import polars as pl

# Path del proyecto
sys.path.append("../src")

from etl.data_loader import load_full_dataset_lazy
from utils.config import CLEANED_PARQUET_FILE

pl.Config.set_tbl_cols(20)
pl.Config.set_fmt_str_lengths(50)

print("✅ Entorno configurado.")

✅ Entorno configurado.


## 1. Limpieza masiva (Lazy + sink_parquet)

In [21]:
print("\n🚀 INICIANDO PROCESO ETL (Modo Streaming Optimizado)...")
print("=" * 60)

# 1. Definir el plan (Lazy)
lf = load_full_dataset_lazy()
print("📡 Conexión Lazy establecida.")

# A) Limpieza de comillas en todas las columnas string
lf = lf.with_columns(
    pl.col(pl.String).str.replace_all('"', "", literal=True)
)

# B) Eliminar columnas 100 % nulas / obsoletas
cols_to_drop = ["FECHA_APERTURA_TEXTO", "NUMERO_CATASTRAL", "NUMERO_CATASTRAL_ANTIGUO"]
existing_drop = [c for c in cols_to_drop if c in lf.columns]
if existing_drop:
    print(f"🗑 Eliminando columnas: {existing_drop}")
    lf = lf.drop(existing_drop)

# C) Limpieza / imputación ORIP (Lazy)
if "ORIP" in lf.columns:
    lf = lf.with_columns(
        pl.when(
            pl.col("ORIP").is_null()
            | (pl.col("ORIP").cast(pl.String).str.strip_chars().is_in(["", "0", "0.0"]))
        )
        .then(pl.lit("SIN_ORIP"))
        .otherwise(
            pl.col("ORIP")
            .cast(pl.String, strict=False)
            .str.strip_chars()
            .str.to_uppercase()
        )
        .alias("ORIP")
    )

# D) Convertir strings vacíos a NULL
lf = lf.with_columns(
    pl.when(pl.col(pl.String).str.strip_chars().str.len_chars() == 0)
    .then(None)
    .otherwise(pl.col(pl.String))
    .name.keep()
)

# E) Conversión de VALOR (str "5,000,000" -> f64 5000000.0)
if "VALOR" in lf.columns:
    lf = lf.with_columns(
        pl.col("VALOR")
        .cast(pl.String)
        .str.replace_all(",", "")
        .str.strip_chars()
        .cast(pl.Float64, strict=False)
        .alias("VALOR")
    )

# 2. Guardar parquet limpio
output_path = Path(CLEANED_PARQUET_FILE)
output_path.parent.mkdir(parents=True, exist_ok=True)

print("\n⏳ Procesando y guardando en disco (sink_parquet)...")
lf.sink_parquet(output_path, compression="zstd", maintain_order=False)

print(f"\n✅ PROCESO FINALIZADO. Archivo guardado en: {output_path}")
print("\n🔎 Vista rápida de los datos limpios:")
print(pl.read_parquet(output_path, n_rows=5))



🚀 INICIANDO PROCESO ETL (Modo Streaming Optimizado)...
Cargando dataset en modo lazy (sin cargar en memoria)...
✓ Dataset cargado en modo lazy
📡 Conexión Lazy establecida.
🗑 Eliminando columnas: ['FECHA_APERTURA_TEXTO', 'NUMERO_CATASTRAL', 'NUMERO_CATASTRAL_ANTIGUO']

⏳ Procesando y guardando en disco (sink_parquet)...


C:\Users\carlo\AppData\Local\Temp\ipykernel_2436\3940625729.py:15: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  existing_drop = [c for c in cols_to_drop if c in lf.columns]
C:\Users\carlo\AppData\Local\Temp\ipykernel_2436\3940625729.py:21: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "ORIP" in lf.columns:
C:\Users\carlo\AppData\Local\Temp\ipykernel_2436\3940625729.py:46: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "VALOR" in lf.columns:



✅ PROCESO FINALIZADO. Archivo guardado en: c:\Users\carlo\Documents\KrozFu\repos\challenge-opendata\igac-anomalias-inmobiliarias\notebooks\..\data\processed\igac_cleaned.parquet

🔎 Vista rápida de los datos limpios:
shape: (5, 23)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ PK  ┆ MAT ┆ FEC ┆ YEA ┆ ORI ┆ DIV ┆ DEP ┆ MUN ┆ TIP ┆ CAT ┆ … ┆ Din ┆ COD ┆ NOM ┆ DOC ┆ COU ┆ COU ┆ PRE ┆ TIE ┆ TIE ┆ VAL │
│ --- ┆ RIC ┆ HA_ ┆ R_R ┆ P   ┆ IPO ┆ ART ┆ ICI ┆ O_P ┆ EGO ┆   ┆ ámi ┆ _NA ┆ BRE ┆ UME ┆ NT_ ┆ NT_ ┆ DIO ┆ NE_ ┆ NE_ ┆ OR  │
│ str ┆ ULA ┆ RAD ┆ ADI ┆ --- ┆ LA  ┆ AME ┆ PIO ┆ RED ┆ RIA ┆   ┆ ca_ ┆ TUJ ┆ _NA ┆ NTO ┆ A   ┆ DE  ┆ S_N ┆ VAL ┆ MAS ┆ --- │
│     ┆ --- ┆ ICA ┆ CA  ┆ str ┆ --- ┆ NTO ┆ --- ┆ IO_ ┆ _RU ┆   ┆ Inm ┆ UR  ┆ TUJ ┆ _JU ┆ --- ┆ --- ┆ UEV ┆ OR  ┆ _DE ┆ f64 │
│     ┆ str ┆ _TE ┆ --- ┆     ┆ i64 ┆ --- ┆ str ┆ ZON ┆ RAL ┆   ┆ obi ┆ --- ┆ UR  ┆ STI ┆ i64 ┆ i64 ┆ OS  ┆ --- ┆ _UN ┆     │
│     ┆     

## 2. Reporte final y dataset base para ML

In [22]:
print("\n📊 REPORTE FINAL DE CALIDAD (Desde disco)")
print("=" * 40)

# Dataset limpio (Lazy)
lf_final = pl.scan_parquet(output_path)
filas_finales = lf_final.select(pl.len()).collect().item()

# Filas iniciales desde origen
lf_origen = load_full_dataset_lazy()
filas_iniciales = lf_origen.select(pl.len()).collect().item()

print(f"Filas Iniciales : {filas_iniciales:,}")
print(f"Filas Finales   : {filas_finales:,}")
print(f"Retención       : {(filas_finales/filas_iniciales)*100:.2f}%")

print("\n🔎 Muestra de datos limpios:")
print(pl.read_parquet(output_path, n_rows=5))

# Estadísticas de nulos
print("\n📉 Estadísticas de Nulos Finales:")
cols = lf_final.collect_schema().names()
null_stats = lf_final.select(
    [pl.col(c).null_count().alias(c) for c in cols]
).collect(engine="streaming")

null_df = null_stats.transpose(include_header=True, header_name="Columna", column_names=["Nulos"])
null_df = null_df.filter(pl.col("Nulos") > 0).sort("Nulos", descending=True)
print(null_df)

# (Opcional) Crear dataset base para ML: filtrar ESTADO_FOLIO, VALOR y eliminar FOLIOS_DERIVADOS
ML_PARQUET_FILE = Path("../data/processed/igac_ml_base.parquet")

lf_ml = lf_final

# Eliminar filas con ESTADO_FOLIO nulo
if "ESTADO_FOLIO" in lf_ml.columns:
    lf_ml = lf_ml.filter(pl.col("ESTADO_FOLIO").is_not_null())

# Eliminar FOLIOS_DERIVADOS si existe
if "FOLIOS_DERIVADOS" in lf_ml.columns:
    lf_ml = lf_ml.drop("FOLIOS_DERIVADOS")

# Mantener solo filas con VALOR útil
if "VALOR" in lf_ml.columns:
    lf_ml = lf_ml.filter(pl.col("VALOR").is_not_null() & (pl.col("VALOR") > 0))

ML_PARQUET_FILE.parent.mkdir(parents=True, exist_ok=True)
print("\n💾 Guardando dataset base para ML...")
lf_ml.sink_parquet(ML_PARQUET_FILE, compression="zstd", maintain_order=False)
print(f"✅ Dataset base para ML guardado en: {ML_PARQUET_FILE}")

print("\n🔎 Muestra de datos para ML:")
print(pl.read_parquet(ML_PARQUET_FILE, n_rows=5))


📊 REPORTE FINAL DE CALIDAD (Desde disco)
Cargando dataset en modo lazy (sin cargar en memoria)...
✓ Dataset cargado en modo lazy
Filas Iniciales : 30,903,248
Filas Finales   : 30,903,248
Retención       : 100.00%

🔎 Muestra de datos limpios:
shape: (5, 23)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ PK  ┆ MAT ┆ FEC ┆ YEA ┆ ORI ┆ DIV ┆ DEP ┆ MUN ┆ TIP ┆ CAT ┆ … ┆ Din ┆ COD ┆ NOM ┆ DOC ┆ COU ┆ COU ┆ PRE ┆ TIE ┆ TIE ┆ VAL │
│ --- ┆ RIC ┆ HA_ ┆ R_R ┆ P   ┆ IPO ┆ ART ┆ ICI ┆ O_P ┆ EGO ┆   ┆ ámi ┆ _NA ┆ BRE ┆ UME ┆ NT_ ┆ NT_ ┆ DIO ┆ NE_ ┆ NE_ ┆ OR  │
│ str ┆ ULA ┆ RAD ┆ ADI ┆ --- ┆ LA  ┆ AME ┆ PIO ┆ RED ┆ RIA ┆   ┆ ca_ ┆ TUJ ┆ _NA ┆ NTO ┆ A   ┆ DE  ┆ S_N ┆ VAL ┆ MAS ┆ --- │
│     ┆ --- ┆ ICA ┆ CA  ┆ str ┆ --- ┆ NTO ┆ --- ┆ IO_ ┆ _RU ┆   ┆ Inm ┆ UR  ┆ TUJ ┆ _JU ┆ --- ┆ --- ┆ UEV ┆ OR  ┆ _DE ┆ f64 │
│     ┆ str ┆ _TE ┆ --- ┆     ┆ i64 ┆ --- ┆ str ┆ ZON ┆ RAL ┆   ┆ obi ┆ --- ┆ UR  ┆ STI ┆ i64 ┆ i64 ┆ OS  ┆ --- 

C:\Users\carlo\AppData\Local\Temp\ipykernel_2436\2232082062.py:36: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "ESTADO_FOLIO" in lf_ml.columns:
C:\Users\carlo\AppData\Local\Temp\ipykernel_2436\2232082062.py:40: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "FOLIOS_DERIVADOS" in lf_ml.columns:
C:\Users\carlo\AppData\Local\Temp\ipykernel_2436\2232082062.py:44: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "VALOR" in lf_ml.columns:


✅ Dataset base para ML guardado en: ..\data\processed\igac_ml_base.parquet

🔎 Muestra de datos para ML:
shape: (5, 22)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ PK  ┆ MAT ┆ FEC ┆ YEA ┆ ORI ┆ DIV ┆ DEP ┆ MUN ┆ TIP ┆ CAT ┆ … ┆ Din ┆ COD ┆ NOM ┆ DOC ┆ COU ┆ COU ┆ PRE ┆ TIE ┆ TIE ┆ VAL │
│ --- ┆ RIC ┆ HA_ ┆ R_R ┆ P   ┆ IPO ┆ ART ┆ ICI ┆ O_P ┆ EGO ┆   ┆ ámi ┆ _NA ┆ BRE ┆ UME ┆ NT_ ┆ NT_ ┆ DIO ┆ NE_ ┆ NE_ ┆ OR  │
│ str ┆ ULA ┆ RAD ┆ ADI ┆ --- ┆ LA  ┆ AME ┆ PIO ┆ RED ┆ RIA ┆   ┆ ca_ ┆ TUJ ┆ _NA ┆ NTO ┆ A   ┆ DE  ┆ S_N ┆ VAL ┆ MAS ┆ --- │
│     ┆ --- ┆ ICA ┆ CA  ┆ str ┆ --- ┆ NTO ┆ --- ┆ IO_ ┆ _RU ┆   ┆ Inm ┆ UR  ┆ TUJ ┆ _JU ┆ --- ┆ --- ┆ UEV ┆ OR  ┆ _DE ┆ f64 │
│     ┆ str ┆ _TE ┆ --- ┆     ┆ i64 ┆ --- ┆ str ┆ ZON ┆ RAL ┆   ┆ obi ┆ --- ┆ UR  ┆ STI ┆ i64 ┆ i64 ┆ OS  ┆ --- ┆ _UN ┆     │
│     ┆     ┆ XTO ┆ i64 ┆     ┆     ┆ str ┆     ┆ A   ┆ IDA ┆   ┆ lia ┆ i64 ┆ --- ┆ FIC ┆     ┆     ┆ --- ┆ i64 ┆ _VA ┆     │